In [4]:
%%html
<style>
body {
    font-family: "Monaco"
}
</style> 

In [2]:
# Use below code snippet to add image
# ![image](image/Decision_Tree_02.png)

# Decision Tree

Decision tree learning algorithms generate decision trees from training data to approximate solutions to classification or regression problems. In the case of classification, trees are typically represented by a set of if-then rules culminating in a decision:

![image](image/Decision_Tree_01.png)

To make this tree, a decision tree learning algorithm would take training data containing various permutations of these four variables and their classifications (*yes, eat there or no, don’t eat there*) and try to produce a tree that is consistent with that data. 

This document will cover classification trees by introducing one of the classic small data sets in the literature: the PlayTennis data set. Then, a popular algorithm used to take training data and produce a decision tree, **the ID3 algorithm**, will be discussed in detail. Finally, we will discuss potential pitfalls when using the data on real data sets and explain workarounds and solutions to them.

## Introduction

Let’s pretend for a moment that you like to play tennis. On a particular day—say, a random Sunday morning—how would you decide whether or not you would head to the nearest tennis court for a few sets? Perhaps you would look outside and check to see if it’s cloudy or raining. Maybe you’d even step outside to see how hot (or cold) it is. Then, you’d use all of this information to inform your decision. If you took that even further, you could record the choices you made on different days (and all of the variables you took into account to make them) into a table like the one below:

![image](image/Decision_Tree_02.png)

With this table, other people would be able to use your intuition to decide whether they should play tennis by looking up what you did given a certain weather pattern, but after just 14 days, it’s a little unwieldy to match your current weather situation with one of the rows in the table. A decision tree would be a great way to represent data like this, because the data takes certain weather patterns and classifies them into a few buckets: tennisplaying weather and not tennisplaying weather. A decision tree for this data allows you to make a decision by following a graph, rather than by looking up your particular situation in a table:

![image](image/Decision_Tree_03.png)

In this case, you’re asked a number of questions about your current weather situation that will result in a yes (let’s play tennis!) or no (let’s stay indoors) answer. So, later we will check **ID3 algorithm** how did this tree result from the training data?

## ID3 Algorithm - Build Decision Tree

The ID3 algorithm builds decision trees using a topdown, greedy approach. Briefly, the steps to the algorithm are: 

1. Start with a training data set, which we’ll call S. It should have attributes and classifications. The attributes of PlayTennis are outlook, temperature humidity, and wind, and the classification is whether or not to play tennis. There are 14 observations. 

2. Determine the best attribute in the data set S. The first attribute ID3 picks in our example is outlook. We’ll go over the definition of “best attribute” shortly. 

3. Split S into subsets that correspond to the possible values of the best attribute. Under outlook, the possible values are sunny, overcast, and rain, so the data is split into three subsets (rows 1, 2, 8, 9, and 11 for sunny; rows 3, 7, 12, and 13 for overcast; and rows 4, 5, 6, 10, and 14 for rain). 

4. Make a decision tree node that contains the best attribute. The outlook attribute takes its rightful place at the root of the PlayTennis decision tree. 

5. Recursively make new decision tree nodes with the subsets of data created in step 3. Attributes can’t be reused. If a subset of data agrees on the classification, choose that classification. If there are no more attributes to split on, choose the most popular classification. The sunny data is split further on humidity because ID3 decides that within the set of sunny rows (1, 2, 8, 9, and 11), humidity is the best attribute. The two paths result in consistent classifications—sunny/high humidity always leads to no and sunny/normal humidity always leads to yes—so the tree ends after that. The rain data behaves in a similar manner, except with the wind attribute instead of the humidity attribute. On the other hand, the overcast data always leads to yes without the help of an additional attribute, so the tree ends immediately.

### Pseudocode:

This pseudocode assumes that the **attributes are discrete and that the classifications are either yes or no. It deals with inconsistent training data by choosing the most popular classification label whenever a possible conflict arises.**

```python
def id3(examples, classification_attribute, attributes):
	create a root node for the tree
	if all examples are positive/yes:
		return root node with positive/yes label
	else if all examples are negative/no:
		return root node with negative/no label
	else if there are no attributes left:
		return root node with most popular
```

```python
classification_attribute label:
	else:
		best_attribute = attribute from attributes that best classifies examples
		assign best_attribute to root node
		
		for each value in best_attribute:
			add branch below root node for the value
			branch_examples = [examples that have that value for best_attribute]
			
			if branch_examples is empty:
				add leaf node with most popular classification_attribute label
			else:
				add subtree id3(branch_examples, classification_attribute, attributes - best_attribute)
```

### Details of ID3 Algorithm

If there’s an attribute for the data to be split on, **the algorithm calls itself recursively, with the original set of examples being split into groups based on the value of the best attribute and the set of available attributes to split on having the best attribute removed from it**. Because this algorithm is a recursive one, the base cases: all examples having the same classification, no attributes being left, or no examples remaining, are tested first.

First, the ID3 algorithm answers the question, “are we done yet?” Being done, in the sense of the ID3 algorithm, means one of two things: 

1. All of the data points to the same classification. This allows ID3 to make a final decision, since all of the training data will agree with it. 

2. There are no more attributes available to divide the data. ID3 only uses each attribute a maximum of one time per path through the tree3. Once it’s reached that maximum, if the remaining data doesn’t point to the same classification, the algorithm is forced to make a final decision, which usually ends up being the most popular classification, because it can’t split up the data any longer.

As you may have guessed, neither of these situations should apply at the very beginning. It would not be particularly useful to have an entire training set with the same classification. Imagine that our PlayTennis data set always told us to play tennis, regardless of the weather situation. Because ID3 would recognize that all of the data points to the same classification and, therefore, it could arrive at a final decision i.e. "YES"

Similarly, it wouldn’t be useful to have an entire training set that has no attributes. If our algorithm could only use classifications and, consequently, had no information except for the ratio of yes rows to no rows, it would be forced to pick the one that’s most popular. There are 9 yes rows and 5 no rows, so the entire decision tree is "YES" again.

Luckily, in this case, the training data is split into 9 yes rows and 5 no rows and there are four potential attributes we can use to split the data. The algorithm, sensing that it’s not done yet, asks the following question: “which attribute should we divide the data with?” The answer is that it should divide the data by the best attribute, but what does “best” actually mean?